In [1]:
import pickle
import numpy as np
import random

In [2]:
with open('data/2021_neurips_processed_paper_data.pkl', 'rb') as file:
    paper_data = pickle.load(file)

In [13]:
abstract_list = []
average_score_list = []

for paper in paper_data:
    
    ratings = []
    for rating in paper['rating_list']:
        ratings.append(float(rating.split(':')[0]))
    if len(ratings) == 0:
        continue
    
    if np.mean(ratings) > 6.3:
        average_score_list.append(1)
    else:
        average_score_list.append(0)
        
    #average_score_list.append(np.mean(ratings))
    
    abstract_list.append(paper['abstract'].lower())
    
    
    

In [14]:
np.mean(average_score_list)

0.5184115523465704

In [15]:
from transformers import AutoTokenizer, AutoModel
from transformers import AutoModelForSequenceClassification


tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
model = AutoModelForSequenceClassification.from_pretrained('allenai/scibert_scivocab_uncased', num_labels=2)

# tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
# model = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased', num_labels=2)

Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification we

In [16]:
inputs = tokenizer(abstract_list, max_length=512,padding="max_length", truncation=True)

In [18]:
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [19]:
import torch
from torch.utils.data import TensorDataset

labels = torch.tensor(average_score_list)
input_ids = torch.tensor(inputs['input_ids'])
attention_mask = torch.tensor(inputs['attention_mask'])


train_dataset = TensorDataset(input_ids[500:], attention_mask[500:], labels[500:])
eval_dataset = TensorDataset(input_ids[:500], attention_mask[:500], labels[:500])

# train_dataset = TensorDataset(input_ids[2000:], attention_mask[2000:], labels[2000:])
# eval_dataset = TensorDataset(input_ids[:50], attention_mask[:50], labels[:50])


In [20]:
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer",
                                  num_train_epochs=10,
                                 per_device_train_batch_size=8,
                                per_device_eval_batch_size=8,
                                 evaluation_strategy='epoch')

In [21]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [22]:
from datasets import load_dataset

raw_datasets = load_dataset("imdb")

Reusing dataset imdb (C:\Users\Chris\.cache\huggingface\datasets\imdb\plain_text\1.0.0\2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


  0%|          | 0/3 [00:00<?, ?it/s]

In [23]:
def dummy_data_collector(features):
    batch = {}
    batch['input_ids'] = torch.stack([f[0] for f in features])
    batch['attention_mask'] = torch.stack([f[1] for f in features])
    batch['labels'] = torch.stack([f[2] for f in features])
    
    return batch

In [24]:
from transformers import Trainer

trainer = Trainer(
    model=model, 
    args=training_args, 
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator= dummy_data_collector,
    compute_metrics=compute_metrics,
)

In [25]:
trainer.train()

***** Running training *****
  Num examples = 2270
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1420
C:\Users\Chris\anaconda3\envs\phd\lib\site-packages\torch\nn\parallel\_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
C:\Users\Chris\anaconda3\envs\phd\lib\site-packages\torch\cuda\nccl.py:15: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn('PyTorch is not compiled with NCCL support')


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.688431,0.546000
2,No log,0.695142,0.476000
3,No log,0.698396,0.476000
4,0.707300,0.695470,0.476000
5,0.707300,0.696199,0.524000
6,0.707300,0.692133,0.524000
7,0.707300,0.697148,0.524000
8,0.703300,0.691985,0.524000
9,0.703300,0.693045,0.524000
10,0.703300,0.694812,0.476000


***** Running Evaluation *****
  Num examples = 500
  Batch size = 16
C:\Users\Chris\anaconda3\envs\phd\lib\site-packages\torch\nn\parallel\_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
C:\Users\Chris\anaconda3\envs\phd\lib\site-packages\torch\cuda\nccl.py:15: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn('PyTorch is not compiled with NCCL support')
***** Running Evaluation *****
  Num examples = 500
  Batch size = 16
C:\Users\Chris\anaconda3\envs\phd\lib\site-packages\torch\nn\parallel\_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
C:\Users\Chris\anaconda3\envs\phd\lib\site-packages\torch\cuda\nccl.py:15: UserWarning: PyTorch i

TrainOutput(global_step=1420, training_loss=0.7047411206742408, metrics={'train_runtime': 951.5843, 'train_samples_per_second': 23.855, 'train_steps_per_second': 1.492, 'total_flos': 5972620956672000.0, 'train_loss': 0.7047411206742408, 'epoch': 10.0})

In [ ]:
trainer.evaluate()

# BAD CODE

In [36]:
shuffled_list = [(x,y) for x,y in zip(abstract_list, average_score_list)]
random.shuffle(shuffled_list)


train_eval_split = .8
train_num = int(np.floor(len(shuffled_list)*train_eval_split))
train_dataset = shuffled_list[:train_num]
eval_dataset = shuffled_list[train_num:]
print('len train_dataset:', len(train_dataset))
print('len eval_dataset:', len(eval_dataset))

from torch.utils.data.dataset import Dataset

class Abstract_Dataset(Dataset):
    def __init__(self, data):
        self.abstracts = []
        self.labels = []
        
        for a,b in data:
            self.abstracts.append(a)
            self.labels.append(b)
        
    def __getitem__(self, index):
        
        return (self.abstracts[index], self.labels[index])

    def __len__(self):
        return len(self.abstracts)

len train_dataset: 2216
len eval_dataset: 554


In [37]:
Train_Dataset = Abstract_Dataset(train_dataset)
Eval_Dataset = Abstract_Dataset(eval_dataset)